In [37]:
import matplotlib.pyplot as plt
import os
import networkx as nx
import pandas as pd
import csv
from collections import *
import numpy as np

In [2]:
os.chdir('./facebook_large/')

In [131]:
def read_csv(csv_path, label_path):
    edges = pd.read_csv(csv_path)
    label = pd.read_csv(label_path)
    edges = edges.rename(columns={"id_1": "source", "id_2": "target"})
    Graphtype = nx.Graph()
    graph = nx.from_pandas_edgelist(edges, create_using=Graphtype)
    for i, node in enumerate(graph.nodes()):
        graph.nodes[node]['pagename'] = label[label['id'] == node]['pagename'][node]
        graph.nodes[node]['pagetype'] = label[label['id'] == node]['pagetype'][node]
    return graph

In [10]:
csv_path = "./musae_facebook_edges.csv"
label_path = "./musae_facebook_target.csv"
Facebook = read_csv(csv_path, label_path)

NameError: name 'read_csv' is not defined

In [11]:
label_path = "./musae_facebook_target.csv"
label = pd.read_csv(label_path)

In [12]:
label.head()

,id,facebook_id,page_name,page_type
0,0,145647315578475,The Voice of China 中国好声音,tvshow
1,1,191483281412,U.S. Consulate General Mumbai,government
2,2,144761358898518,ESET,company
3,3,568700043198473,Consulate General of Switzerland in Montreal,government
4,4,1408935539376139,Mark Bailey MP - Labor for Miller,politician


In [133]:
degree = Facebook.degree()

In [134]:
b_centrality = nx.betweenness_centrality(Facebook)

In [135]:
for n in Facebook.nodes():
    Facebook.nodes[n]['degree'] = degree[n]
for n in Facebook.nodes():
    Facebook.nodes[n]['between'] = b_centrality[n]

In [146]:
# for n in Facebook:
#     if 'page_name' in Facebook.nodes[n]:
#         Facebook.nodes[n]['pagename'] = Facebook.nodes[n]['page_name']
#         del Facebook.nodes[n]['page_name']
#     if 'page_type' in Facebook.nodes[n]:
#         Facebook.nodes[n]['pagetype'] = Facebook.nodes[n]['page_type']
#         del Facebook.nodes[n]['page_type']

In [145]:
Facebook.nodes[0]

{'degree': 1,
 'between': 0.0,
 'pagename': 'The Voice of China 中国好声音',
 'pagetype': 'tvshow'}

In [147]:
nx.write_gml(Facebook, "./facebook.gml")

In [5]:
Facebook = nx.read_gml('./facebook.gml')

In [6]:
import community

In [7]:
def getCommunity(graph):
    partitions = community.best_partition(graph, randomize=False)
    for n in graph.nodes:
        graph.nodes[n]['community'] = partitions[n]
    return partitions, graph
fb_community, Facebook = getCommunity(Facebook)


In [51]:
nx.write_gml(Facebook, "./facebook.gml")

In [15]:
len(set(fb_community.values()))

67

In [20]:
count = Counter(fb_community.values())

In [45]:
five_community = [comm for comm in count.keys() if count[comm] in sorted(count.values(), reverse=True)[:4]]

In [46]:
Community = collections.defaultdict(list)
Category = collections.defaultdict(list)
for node in Facebook.nodes():
    if Facebook.nodes[node]['community'] in five_community:
        Community[Facebook.nodes[node]['community']].append((Facebook.nodes[node]['degree'], node))
    Category[Facebook.nodes[node]['pagetype']].append((Facebook.nodes[node]['degree'], node)) 
        

In [47]:
table = []
table2= []
# area = {3: 'Graph Theory', 5: "Number Theory", 9: "Graph Theory", 11:"Combinatorics", 16:"Erdos Number"}
for common, ls in Community.items():
    ls = sorted(ls, reverse=True)[:3]
    for degree, node in ls:
        table.append([Facebook.nodes[node]['pagename'], degree, Facebook.nodes[node]['community'], Facebook.nodes[node]['pagetype']])

for common, ls in Category.items():
    ls = sorted(ls, reverse=True)[:3]
    for degree, node in ls:
        table2.append([Facebook.nodes[node]['pagename'], degree, Facebook.nodes[node]['community'], Facebook.nodes[node]['pagetype']])

In [48]:
pd.DataFrame(table, columns=["Page Name", "Degree", "Community", "Page Type"])

,Page Name,Degree,Community,Page Type
0,U.S. Army,709,1,government
1,The White House,678,1,government
2,The Obama White House,659,1,government
3,FEMA Federal Emergency Management Agency,448,3,government
4,U.S. National Weather Service (NWS),296,3,government
5,National Oceanic and Atmospheric Administratio...,288,3,government
6,Today Show,141,13,tvshow
7,Home & Family,137,13,tvshow
8,Walmart,124,13,company
9,European Parliament,417,14,government


In [49]:
pd.DataFrame(table2, columns=["Page Name", "Degree", "Community", "Page Type"])

,Page Name,Degree,Community,Page Type
0,Today Show,141,13,tvshow
1,Home & Family,137,13,tvshow
2,tagesschau,119,21,tvshow
3,U.S. Army,709,1,government
4,The White House,678,1,government
5,The Obama White House,659,1,government
6,Facebook,380,4,company
7,NASA - National Aeronautics and Space Administ...,328,1,company
8,CNN,167,1,company
9,Barack Obama,341,1,politician
